In [1]:
# add parent directory to path: enable import from parent dir
import sys
sys.path.append('../')

from environment import SmartBrokerEnv
from agents.a2c import A2C
from networks.a2c.actor import Actor
from networks.a2c.critic import Critic

import gym
import matplotlib.pyplot as plt
import torch

In [2]:
norm_cols = ['Volume XRP']
cols = ['date'] + ['open', 'high', 'low', 'close', f'Volume XRP']
batch_dur = 15
env = SmartBrokerEnv(
    batch_dur=batch_dur,
    df_info={
        'start_date': '2021-01-01',
        'end_date': '2021-02-01',
        'norm_cols': norm_cols,
        'cols': cols,
    },
    portfolio={

    },
)

/Users/vinay/miniforge3/lib/python3.9/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Users/vinay/code/git/smart-broker/notebooks/../environment.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['rolling_price'] = self.df[self.price_typ].rolling(self.roll_period).sum()
/Users/vinay/miniforge3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [3]:
actor_model = Actor(
    state_dim=batch_dur*3+3,
    action_dim=3,
)
critic_model = Critic(state_dim=batch_dur*3+3)
a2c = A2C(
    env=env, 
    actor=actor_model,
    critic=critic_model,
    n_actns=3,
    actor_optmz=torch.optim.Adam(actor_model.parameters(), lr=0.001),
    critic_optmz=torch.optim.Adam(critic_model.parameters(), lr=0.001),
    hyprprms={
        'gamma': 0.9995,
    },
    log_freq=25,
)

In [4]:
a2c.run(1000)


Ep: 0 | L: 1.173 | R: 53.95 | R.Avg.R: 53.95 | P: 3.58 | R.Avg P: 3 | B: 44.23 | R.Avg B: 44 | R.N_Units: 257
Ep: 25 | L: 0.975 | R: 55.51 | R.Avg.R: 54.71 | P: 6.18 | R.Avg P: 4 | B: 20.59 | R.Avg B: 21 | R.N_Units: 362
Ep: 50 | L: 1.02 | R: 54.7 | R.Avg.R: 54.44 | P: 5.18 | R.Avg P: 4 | B: 26.18 | R.Avg B: 21 | R.N_Units: 360
Ep: 75 | L: 0.988 | R: 53.67 | R.Avg.R: 53.91 | P: 3.18 | R.Avg P: 4 | B: 69.0 | R.Avg B: 30 | R.N_Units: 318
Ep: 100 | L: 1.054 | R: 52.74 | R.Avg.R: 52.4 | P: 1.6 | R.Avg P: 2 | B: 92.22 | R.Avg B: 60 | R.N_Units: 181
Ep: 125 | L: 1.003 | R: 51.53 | R.Avg.R: 52.29 | P: -0.05 | R.Avg P: 1 | B: 85.76 | R.Avg B: 85 | R.N_Units: 66
Ep: 150 | L: 0.991 | R: 52.61 | R.Avg.R: 51.91 | P: 1.38 | R.Avg P: 0 | B: 87.39 | R.Avg B: 87 | R.N_Units: 57
Ep: 175 | L: 0.959 | R: 51.44 | R.Avg.R: 51.98 | P: -0.01 | R.Avg P: 0 | B: 92.8 | R.Avg B: 89 | R.N_Units: 49
Ep: 200 | L: 1.039 | R: 53.21 | R.Avg.R: 52.16 | P: 2.43 | R.Avg P: 0 | B: 89.8 | R.Avg B: 91 | R.N_Units: 39
Ep: 2

KeyboardInterrupt: 

## Visualizations

### Traning 

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(16, 12))
r_avg_rewards = []
r_avg_profits = []
r_avg_bal = []
r_avg_units_held = []
r_avg_loss = []
r_avg_net_worth = []
count = 0

for _, log in a2c.logs.items():
    r_avg_rewards.append(log['r_avg_reward'])
    r_avg_profits.append(log['r_avg_profit'])
    r_avg_bal.append(log['r_avg_bal'])
    r_avg_units_held.append(log['r_avg_units_held'])
    r_avg_loss.append(log['r_avg_loss'])
    r_avg_net_worth.append(log['r_avg_net_worth'])
    count += 1

ax[0][0].plot(range(count), r_avg_loss)
ax[0][0].set_title('Rolling avg loss per episode')

ax[0][1].plot(range(count), r_avg_rewards)
ax[0][1].set_title('Rolling avg reward per episode')

ax[1][0].plot(range(count), r_avg_profits)
ax[1][0].set_title('Rolling avg profit per episode')

ax[1][1].plot(range(count), r_avg_units_held)
ax[1][1].set_title('Rolling avg units held per episode')

ax[2][0].plot(range(count), r_avg_net_worth)
ax[2][0].set_title('Rolling avg net worth per episode')

ax[2][1].plot(range(count), r_avg_bal)
ax[2][1].set_title('Rolling avg balance per episode')

In [ ]:
# import pickle
# with open("../pickles/a2c_logs.pickle","wb") as f:
#     pickle.dump(temp, f, pickle.HIGHEST_PROTOCOL)